In [ ]:
import glob
import pandas as pd

indirs = sorted(glob.glob('/g/data/hj10/admin/cpol_level_1a/v2019/ppi/**/**/'))
dlist = [pd.to_datetime(d[-9:-1]) for d in indirs]
drange = pd.DatetimeIndex(dlist).to_period(freq='1m').unique()

In [ ]:
def get_pbs(sdate, edate):
    txt = f'''#!/bin/bash
#PBS -P kl02
#PBS -q normal
#PBS -l walltime=8:00:00
#PBS -l mem=128GB
#PBS -l wd
#PBS -l jobfs=1GB
#PBS -l ncpus=16
#PBS -lstorage=scratch/hj10+gdata/hj10+scratch/kl02+gdata/kl02
source activate radar
export NUMBA_CACHE_DIR=/scratch/kl02/vhl548/tmp/
python radar_pack.py -s {sdate} -e {edate}
'''
    return txt

In [ ]:
for a, b in [(d.strftime('%Y%m') + '01', d.strftime('%Y%m%d')) for d in drange if d.year >= 2009]:
    t = get_pbs(a, b)
    fname = f'qcpol_{a[:-2]}.pbs'
    
    with open(fname, 'w+') as fid:
        fid.write(t)